In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from maps import map_robinson, map_eq_earth
from GMST import GMST_timeseries, GMST_regression, atm_heat_content, GMST_GISTEMP
from paths import path_results, path_samoc, file_ex_atm_T_T850_U_V_yrly, file_ex_atm_ctrl, CESM_filename, file_ex_atm_lpd
from plotting import shifted_color_map, discrete_cmap
from constants import abs_zero, cp_air
from timeseries import IterateOutputCESM
from xr_integrate import xr_surf_mean, xr_zonal_mean
from xr_DataArrays import xr_AREA

In [ ]:
AREA     = xr_AREA('atm')
AREA_low = xr_AREA('atm_f19')

In [ ]:
ds = xr.open_dataset(file_ex_atm_T_T850_U_V_yrly, decode_times=False)
print(f'The GMST is {xr_surf_mean(ds["T"][-1,:,:], AREA=AREA)+abs_zero:5.2f} deg C.')

# Observations
HadCRUT4
GISTEMP
BEST


In [ ]:
GISTEMP = GMST_GISTEMP()

In [ ]:
plt.axhline(0, c='k', lw=.5)
GISTEMP.GMST.plot()
GISTEMP.GMST.rolling(time=10, center=True).mean().plot()

# CESM runs

In [ ]:
# %%time
# # ca. 30 sec per run
# gmst_ctrl = GMST_timeseries('ctrl')
# gmst_rcp  = GMST_timeseries('rcp')
# gmst_lpd  = GMST_timeseries('lpd')
# gmst_lpd  = GMST_timeseries('lpi')

In [ ]:
gmst_ctrl = xr.open_dataset(f'{path_results}/GMST/GMST_ctrl.nc')
gmst_rcp  = xr.open_dataset(f'{path_results}/GMST/GMST_rcp.nc' )
gmst_lpd  = xr.open_dataset(f'{path_results}/GMST/GMST_lpd.nc' )
gmst_lpi  = xr.open_dataset(f'{path_results}/GMST/GMST_lpi.nc' )

In [ ]:
fig = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot(gmst_ctrl.time/365    , gmst_ctrl.GMST.values, lw=2, label='CTRL')
plt.plot(gmst_rcp.time/365-1800, gmst_rcp.GMST.values,  lw=2, label='RCP')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('GMST [$^\circ$C]', fontsize=16)
plt.legend(fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/GMST/GMST_rcp_ctrl')

In [ ]:
N_ctrl = len(gmst_ctrl.coords['time'])
N_rcp  = len(gmst_rcp.coords['time'])
print(N_ctrl, N_rcp)

fig = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
for i in range(N_rcp):
    plt.plot(gmst_rcp.coords['lat'], 
             gmst_rcp.T_zonal[i,:]-gmst_ctrl.T_zonal[:,:].mean(dim='time'), 
             color=plt.cm.rainbow(i/N_rcp), alpha=.4, lw=.5,
             label='')
for i in range(N_rcp):
    if (i-5)%10==0:
        plt.plot(gmst_rcp.coords['lat'], 
                 gmst_rcp.T_zonal[i-5:i+5,:].mean(dim='time')-gmst_ctrl.T_zonal[:,:].mean(dim='time'), 
                 color=plt.cm.rainbow(i/N_rcp), alpha=1, lw=2.5, ls='--',
                 label=f'decade {int((i-5)/10)+1}')
plt.axhline(0, c='k', lw=.5)
plt.legend(ncol=3, frameon=False, fontsize=16)
plt.ylabel('zonally averaged temp. RCP - CTRL [K]', fontsize=16)
plt.xlabel('Latitude', fontsize=16)
plt.xticks(np.arange(-90,91,30))
plt.tight_layout()
plt.savefig(f'{path_results}/GMST/T_zonal_rcp-ctrl')

## low resolution runs

In [ ]:
plt.plot(gmst_ctrl.time/365+1900, gmst_ctrl.GMST)
plt.plot(gmst_rcp.time/365+ 200 , gmst_rcp.GMST)
plt.plot(gmst_lpd.time/365+1400 , gmst_lpd.GMST)
plt.plot(gmst_lpi.time/365-1600 , gmst_lpi.GMST)

# linear trend maps

In [ ]:
# %%time
# # ca 10:30 min
# trends_ctrl = GMST_regression('ctrl')
# trends_ctrl.to_netcdf(path=f'{path_results}/GMST/trend_ctrl.nc' , mode='w')
# trends_rcp  = GMST_regression('rcp')
# trends_rcp.to_netcdf(path=f'{path_results}/GMST/trend_rcp.nc' , mode='w')

In [ ]:
trends_ctrl = xr.open_dataarray(f'{path_results}/GMST/trend_ctrl.nc')
trends_rcp  = xr.open_dataarray(f'{path_results}/GMST/trend_rcp.nc' )

In [ ]:
times = ['100-299', '2000-2099']
for i, trends in enumerate([trends_ctrl, trends_rcp]):
    run = ['ctrl', 'rcp'][i]
    label = f'{times[i]} air surface temperature trend [K/century]'
    minv, maxv = -2, 6
    cmap = shifted_color_map(mpl.cm.RdBu_r, start=.33, midpoint=0.5, stop=1., name='shrunk')
    cmap = discrete_cmap(16, cmap)
    filename = f'{path_results}/GMST/T_trend_map_{run}'
    map_eq_earth(xa=trends, domain='atm', cmap=cmap, minv=minv, maxv=maxv, label=label, filename=filename)

# Atmospheric Heat Content (AHC)
$$ \mathrm{AHC} = \frac{1}{g} \int \int \int_{0}^{1000hPa} T(p, x, y) dp dx dy$$

In [ ]:
ds = xr.open_dataset(file_ex_atm_ctrl)
dp = (ds.ilev-ds.ilev.shift(ilev=1)).dropna(dim='ilev')
dp = dp.drop(['ilev'])
dp = dp.rename({'ilev':'lev'})
dp = dp.assign_coords(lev=ds.lev)

In [ ]:
%%time
# 100 sec
for run in ['ctrl', 'rcp']:
    print(run)
    for i, (y,m,s) in enumerate(IterateOutputCESM(domain='atm', run=run, tavg='yrly', name='T_T850_U_V')):
        ds = xr.open_dataset(s, decode_times=False)
        da = atm_heat_content(ds, dp, AREA)    
        if i==0:
            da_new = da
        else: 
            da_new = xr.concat([da_new, da], dim='time')
    da_new.to_netcdf(f'{path_results}/GMST/AHC_yrly_{run}.nc')

In [ ]:
AHC_ctrl  = xr.open_dataarray(f'{path_results}/GMST/AHC_yrly_ctrl.nc', decode_times=False)
AHC_rcp   = xr.open_dataarray(f'{path_results}/GMST/AHC_yrly_rcp.nc' , decode_times=False)
dAHC_ctrl = (AHC_ctrl - AHC_ctrl.shift(time=1)).dropna(dim='time')
dAHC_rcp  = (AHC_rcp  - AHC_rcp .shift(time=1)).dropna(dim='time')

In [ ]:
AHC_ctrl[101].time/365

In [ ]:
AHC_0 = AHC_ctrl[101]

f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)

plt.axhline(0, c='k', lw=.5)
plt.axvline(200, c='g', lw=.5)
plt.plot(AHC_ctrl.time/365    , (AHC_ctrl-AHC_0)/1e21)
plt.plot(AHC_rcp .time/365+200, (AHC_rcp -AHC_0)/1e21)
plt.ylabel('AHC anomaly [ZJ]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.tight_layout()

f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)

plt.axhline(0, c='k', lw=.5)
plt.axvline(200, c='g', lw=.5)
plt.plot(dAHC_ctrl.time/365    , dAHC_ctrl/1e21)
plt.plot(dAHC_rcp .time/365+300, dAHC_rcp /1e21)
plt.ylabel('$\Delta$AHC [ZJ/yr]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/GMST/dAHC')

In [ ]:
dGMST_ctrl = (gmst_ctrl.GMST - gmst_ctrl.GMST.shift(time=1)).dropna(dim='time')
dGMST_rcp  = (gmst_rcp .GMST - gmst_rcp .GMST.shift(time=1)).dropna(dim='time')

In [ ]:
plt.axhline(0, c='k', lw=.5)
plt.axvline(0, c='k', lw=.5)
plt.scatter(dGMST_ctrl-dGMST_ctrl.mean(), dAHC_ctrl-dAHC_ctrl.mean())
plt.scatter(dGMST_rcp -dGMST_rcp .mean(), dAHC_rcp -dAHC_rcp .mean())

as expected, positive correlation between $\Delta$AHC and $\Delta$GMST